# Data Science Capstone Week 1

In [1]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn import preprocessing, metrics
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
print("Hello Capstone Project Course!")

Hello Capstone Project Course!


# Capstone Week 2

In [3]:
# The code was removed by Watson Studio for sharing.

/opt/conda/envs/Python36/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (33) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,SEVERITYCODE,X,Y,OBJECTID,INCKEY,COLDETKEY,REPORTNO,STATUS,ADDRTYPE,INTKEY,...,ROADCOND,LIGHTCOND,PEDROWNOTGRNT,SDOTCOLNUM,SPEEDING,ST_COLCODE,ST_COLDESC,SEGLANEKEY,CROSSWALKKEY,HITPARKEDCAR
0,2,-122.323148,47.703140,1,1307,1307,3502005,Matched,Intersection,37475.0,...,Wet,Daylight,NaN,NaN,NaN,10,Entering at angle,0,0,N
1,1,-122.347294,47.647172,2,52200,52200,2607959,Matched,Block,NaN,...,Wet,Dark - Street Lights On,NaN,6354039.0,NaN,11,From same direction - both going straight - bo...,0,0,N
2,1,-122.334540,47.607871,3,26700,26700,1482393,Matched,Block,NaN,...,Dry,Daylight,NaN,4323031.0,NaN,32,One parked--one moving,0,0,N
3,1,-122.334803,47.604803,4,1144,1144,3503937,Matched,Block,NaN,...,Dry,Daylight,NaN,NaN,NaN,23,From same direction - all others,0,0,N
4,2,-122.306426,47.545739,5,17700,17700,1807429,Matched,Intersection,34387.0,...,Wet,Daylight,NaN,4028032.0,NaN,10,Entering at angle,0,0,N


## Check datatypes of imported data

In [4]:
df.dtypes

SEVERITYCODE        int64
X                 float64
Y                 float64
OBJECTID            int64
INCKEY              int64
COLDETKEY           int64
REPORTNO           object
STATUS             object
ADDRTYPE           object
INTKEY            float64
LOCATION           object
EXCEPTRSNCODE      object
EXCEPTRSNDESC      object
SEVERITYCODE.1      int64
SEVERITYDESC       object
COLLISIONTYPE      object
PERSONCOUNT         int64
PEDCOUNT            int64
PEDCYLCOUNT         int64
VEHCOUNT            int64
INCDATE            object
INCDTTM            object
JUNCTIONTYPE       object
SDOT_COLCODE        int64
SDOT_COLDESC       object
INATTENTIONIND     object
UNDERINFL          object
WEATHER            object
ROADCOND           object
LIGHTCOND          object
PEDROWNOTGRNT      object
SDOTCOLNUM        float64
SPEEDING           object
ST_COLCODE         object
ST_COLDESC         object
SEGLANEKEY          int64
CROSSWALKKEY        int64
HITPARKEDCAR       object
dtype: objec

In [5]:
df['SEVERITYDESC'].value_counts()

Property Damage Only Collision    136485
Injury Collision                   58188
Name: SEVERITYDESC, dtype: int64

## Drop features that have no impact on outcome, such as arbitrary report numbers, etc.

In [6]:
df.drop(['OBJECTID', 'INCKEY', 'COLDETKEY', 'REPORTNO', 'EXCEPTRSNCODE', 'EXCEPTRSNDESC', 'INCDATE', 'SEVERITYCODE.1', 'ST_COLDESC'], 1, inplace = True)

## Preprocessing - Review each feature and fill in the NA's.

In [7]:
df['SPEEDING'].fillna('N', inplace =True)
df['SPEEDING'].value_counts()
df['INATTENTIONIND'].fillna('N', inplace =True)
df['INATTENTIONIND'].value_counts()
df['ROADCOND'].fillna('Unknown', inplace = True)
df['ROADCOND'].value_counts()
df['LIGHTCOND'].fillna('Unknown', inplace = True)
df['LIGHTCOND'].value_counts()
df['PEDROWNOTGRNT'].fillna('N', inplace = True)
df['PEDROWNOTGRNT'].value_counts()
df['SDOTCOLNUM'].fillna(0, inplace = True)
df['ST_COLCODE'].fillna(31, inplace = True)
df['JUNCTIONTYPE'].fillna('Unknown', inplace = True)
df['JUNCTIONTYPE'].value_counts()

Mid-Block (not related to intersection)              89800
At Intersection (intersection related)               62810
Mid-Block (but intersection related)                 22790
Driveway Junction                                    10671
Unknown                                               6338
At Intersection (but not related to intersection)     2098
Ramp Junction                                          166
Name: JUNCTIONTYPE, dtype: int64

## Checking how many more features have NA's

In [8]:
df.isnull().sum(axis = 0)

SEVERITYCODE           0
X                   5334
Y                   5334
STATUS                 0
ADDRTYPE            1926
INTKEY            129603
LOCATION            2677
SEVERITYDESC           0
COLLISIONTYPE       4904
PERSONCOUNT            0
PEDCOUNT               0
PEDCYLCOUNT            0
VEHCOUNT               0
INCDTTM                0
JUNCTIONTYPE           0
SDOT_COLCODE           0
SDOT_COLDESC           0
INATTENTIONIND         0
UNDERINFL           4884
WEATHER             5081
ROADCOND               0
LIGHTCOND              0
PEDROWNOTGRNT          0
SDOTCOLNUM             0
SPEEDING               0
ST_COLCODE             0
SEGLANEKEY             0
CROSSWALKKEY           0
HITPARKEDCAR           0
dtype: int64

In [9]:
df['ADDRTYPE'].fillna('Block', inplace = True)
df['ADDRTYPE'].value_counts()
df['WEATHER'].fillna('Other', inplace = True)
df['WEATHER'].value_counts()
df['UNDERINFL'].replace({'N':'0', 'Y':'1'}, inplace = True)
df['UNDERINFL'].value_counts()
df['COLLISIONTYPE'].fillna('Other', inplace = True)
df['COLLISIONTYPE'].value_counts()

Parked Car    47987
Angles        34674
Rear Ended    34090
Other         28607
Sideswipe     18609
Left Turn     13703
Pedestrian     6608
Cycles         5415
Right Turn     2956
Head On        2024
Name: COLLISIONTYPE, dtype: int64

## Filling in the remaining NA's with '0'

In [10]:
df.fillna(0, inplace = True)
df.dtypes

SEVERITYCODE        int64
X                 float64
Y                 float64
STATUS             object
ADDRTYPE           object
INTKEY            float64
LOCATION           object
SEVERITYDESC       object
COLLISIONTYPE      object
PERSONCOUNT         int64
PEDCOUNT            int64
PEDCYLCOUNT         int64
VEHCOUNT            int64
INCDTTM            object
JUNCTIONTYPE       object
SDOT_COLCODE        int64
SDOT_COLDESC       object
INATTENTIONIND     object
UNDERINFL          object
WEATHER            object
ROADCOND           object
LIGHTCOND          object
PEDROWNOTGRNT      object
SDOTCOLNUM        float64
SPEEDING           object
ST_COLCODE         object
SEGLANEKEY          int64
CROSSWALKKEY        int64
HITPARKEDCAR       object
dtype: object

## Handling the non-numerical data

In [11]:
def handle_non_numerical_data(df):
    columns = df.columns.values
    
    for column in columns:
        text_digit_vals = {}
        def convert_to_int(val):
            return text_digit_vals[val]
        
        if df[column].dtype != np.int64 and df[column].dtype != np.float64:
            column_contents = df[column].values.tolist()
            unique_elements = set(column_contents)
            x = 0
            for unique in unique_elements:
                if unique not in text_digit_vals:
                    text_digit_vals[unique] = x
                    x += 1
            df[column] = list(map(convert_to_int, df[column]))
    return df
df = handle_non_numerical_data(df)
df.head

<bound method NDFrame.head of         SEVERITYCODE           X          Y  STATUS  ADDRTYPE   INTKEY  \
0                  2 -122.323148  47.703140       0         0  37475.0   
1                  1 -122.347294  47.647172       0         1      0.0   
2                  1 -122.334540  47.607871       0         1      0.0   
3                  1 -122.334803  47.604803       0         1      0.0   
4                  2 -122.306426  47.545739       0         0  34387.0   
5                  1 -122.387598  47.690575       0         0  36974.0   
6                  1 -122.338485  47.618534       0         0  29510.0   
7                  2 -122.320780  47.614076       0         0  29745.0   
8                  1 -122.335930  47.611904       0         1      0.0   
9                  2 -122.384700  47.528475       0         0  34679.0   
10                 1    0.000000   0.000000       0         2      0.0   
11                 1 -122.333831  47.547371       0         0  33194.0   
12      

In [12]:
df['SEVERITYCODE'].value_counts() #Even though the metadata description shows 5 codes to classify severity, how many are actually present in the data?

1    136485
2     58188
Name: SEVERITYCODE, dtype: int64

## Setting up the Decision Tree Classifier

In [13]:
X = np.array(df.drop(['SEVERITYCODE'],1).astype(float))
y = np.array(df['SEVERITYCODE'])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3)
clf = DecisionTreeClassifier(criterion = "entropy", max_depth = 4)
clf.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=4,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [14]:
prediction = clf.predict(X_test)

In [15]:
print("DecisionTrees's Accuracy: ", metrics.accuracy_score(y_test, prediction)) #Perfect accuracy!!! What!?!?!?

DecisionTrees's Accuracy:  1.0
